OBS: EXECUTAR ESTE NOTEBOOK NO GOOGLE COLAB PARA EVITAR PROBLEMAS COM AS CONFIGURAÇÕES DO AMBIENTE DE EXECUÇÃO

Instalando bibliotecas

In [51]:
# !pip install pycaret[full]
# !pip install pyarrow

Importando bibliotecas

In [52]:
import pandas as pd
from pycaret.time_series import *

Importano base de dados

In [53]:
df = pd.read_parquet(r'df_consolidado_diario.parquet')
df.reset_index(drop = False, inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
df.head(3)

,period,bx_gse,by_gse,bz_gse,bt,density,speed,temperature,source,gse_x_satellite,gse_y_satellite,gse_z_satellite,smoothed_ssn,dst
date,,,,,,,,,,,,,,
1998-02-16,train_a,-5.55,3.00,1.25,6.800000,1.53,383.920013,110237.0,ac,1522376.9,143704.6,149496.7,65.400002,-7.0
1998-02-17,train_a,-5.69,0.87,0.95,6.160000,2.34,366.700012,39783.0,ac,1525410.9,136108.8,151034.1,65.907692,-7.0
1998-02-18,train_a,-5.13,14.41,-5.86,18.120001,12.24,414.420013,120619.0,ac,1528484.9,128470.5,152387.7,66.415382,-100.0


In [54]:
df_a = df[['dst']][df['period'] == 'train_a']
df_a.to_parquet('df_a.parquet')
df_b = df[['dst']][df['period'] == 'train_b']
df_b.to_parquet('df_b.parquet')
df_c = df[['dst']][df['period'] == 'train_c']
df_c.to_parquet('df_c.parquet')
df_a.head(3)

,dst
date,
1998-02-16,-7.0
1998-02-17,-7.0
1998-02-18,-100.0


In [55]:
df_intervalo_a_c = pd.read_parquet(r'df_intervalo_a_c.parquet')
df_intervalo_a_c = df_intervalo_a_c.reset_index(drop = False)
df_intervalo_a_c = df_intervalo_a_c.rename(columns={'index': 'date',
                                                    'dst': 'dst'})
df_intervalo_a_c['date'] = df_intervalo_a_c['date'].dt.to_timestamp()
df_intervalo_a_c['date'] = pd.to_datetime(df_intervalo_a_c['date'])
df_intervalo_a_c = df_intervalo_a_c.set_index('date')
df_intervalo_a_c

,dst
date,
2001-05-29,6.3163
2001-05-30,12.4855
2001-05-31,6.4557
2001-06-01,12.5595
2001-06-02,6.5173
...,...
2004-04-24,43.6373
2004-04-25,37.5946
2004-04-26,43.6961


In [56]:
df_completo = pd.concat([df_a, df_intervalo_a_c, df_c])
df_completo

,dst
date,
1998-02-16,-7.0
1998-02-17,-7.0
1998-02-18,-100.0
1998-02-19,-43.0
1998-02-20,-50.0
...,...
2010-12-27,-10.0
2010-12-28,-11.0
2010-12-29,-25.0


In [57]:
df_completo = df_completo.reset_index(drop = False)
len(df_completo['date'].unique())

4700

In [58]:
df_completo = df_completo.drop_duplicates(subset=['date'], keep='first')
len(df_completo['date'].unique())

4700

In [59]:
# df_completo = df_completo.set_index('date')
# df_completo.head(3)

In [64]:
# df_completo = df_completo.set_index('date').sort_index()

# Infer the frequency of your time series data
freq = pd.infer_freq(df_completo.index)

# If frequency couldn't be inferred, you might need to set it manually
# For example, if you know it's daily data:
if freq is None:
    freq = 'D'

# Reindex the DataFrame with the inferred or set frequency
df_completo = df_completo.asfreq(freq)

# Now set up your time series analysis
setup = setup(df_completo,
              fh = 3,
              session_id = 42,
              fold_strategy = 'expanding',
              numeric_imputation_target = "nearest")

,Description,Value
0,session_id,42
1,Target,dst
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(4702, 1)"
5,Transformed data shape,"(4702, 1)"
6,Transformed train set shape,"(4699, 1)"
7,Transformed test set shape,"(3, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


Inicializando o ambiente de treinamento e criando o pipeline de transformações

Treinando e avaliando modelos com validação cruzada

In [65]:
best = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,0.2912,0.2002,3.9659,4.7602,0.7369,0.5374,-7.4199,1.5233
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,0.2938,0.1938,4.0024,4.6087,0.7137,0.5272,-5.0541,7.9833
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,0.3052,0.2141,4.1576,5.0905,0.8321,0.4801,-6.4110,0.2867
lasso_cds_dt,Lasso w/ Cond. Deseasonalize & Detrending,0.3052,0.2141,4.1575,5.0906,0.8321,0.4800,-6.4117,0.2767
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,0.3076,0.2154,4.1895,5.1212,0.8360,0.4838,-6.6098,0.2967
lr_cds_dt,Linear w/ Cond. Deseasonalize & Detrending,0.3101,0.2167,4.2236,5.1526,0.8405,0.4878,-6.8151,0.9667
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,0.3101,0.2167,4.2236,5.1526,0.8405,0.4878,-6.8151,0.2833
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,0.3102,0.2167,4.2259,5.1509,0.8398,0.4885,-6.8245,0.2900
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,0.3158,0.2050,4.3021,4.8733,0.7445,0.5636,-7.4807,10.9633
stlf,STLF,0.3170,0.2457,4.3178,5.8408,0.9964,0.5494,-7.6813,0.3367


Processing:   0%|          | 0/121 [00:00<?, ?it/s]

Criando modelo selecionado

In [66]:
best_model = create_model('catboost_cds_dt',
                          fold=5)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2010-12-13,0.7398,0.4639,10.0846,11.0366,0.5860,0.8750,-7.9857
1,2010-12-16,0.2705,0.1628,3.6879,3.8716,0.3392,0.3197,-0.2971
2,2010-12-19,0.5444,0.3641,7.4173,8.6579,1.5245,0.9342,0.0166
3,2010-12-22,0.2160,0.1265,2.9427,3.0077,0.4607,0.4696,-9.1767
4,2010-12-25,0.1210,0.0909,1.6472,2.1606,0.1557,0.1778,-6.0022
Mean,NaT,0.3784,0.2416,5.1559,5.7469,0.6132,0.5553,-4.6890
SD,NaT,0.2291,0.1460,3.1232,3.4738,0.4773,0.3004,3.8513


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Otimizando hiperparametros

In [68]:
tuned_model, tuner = tune_model(best_model,
                                return_tuner=True,
                                search_algorithm = "random",
                                optimize="RMSE",
                                )
tuned_model

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2010-12-19,0.5433,0.3740,7.4015,8.8939,1.5563,0.9182,-0.0378
1,2010-12-22,0.2165,0.1507,2.9490,3.5835,0.5303,0.3797,-13.4468
2,2010-12-25,0.2094,0.1214,2.8506,2.8853,0.2869,0.3368,-11.4877
Mean,NaT,0.3230,0.2154,4.4004,5.1209,0.7911,0.5449,-8.3241
SD,NaT,0.1558,0.1128,2.1225,2.6831,0.5501,0.2645,5.9137


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  4.2min finished


BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [18, 17,
                                                                        16, 15,
                                                                        14, 13,
                                                                        12, 11,
                                                                        10, 9,
                                                                        8, 7, 6,
                                                                        5, 4, 3,
                                                                        2, 1]},
                                                   n_jobs=1)],
                    regressor=<catboost.core.CatBoostRegressor object at 0x7941cbae96f0>,
                    sp=18, window_length=18)

Visualizando predição

In [69]:
plot_model(tuned_model, plot = 'forecast')

Visualizando residuos

In [70]:
plot_model(tuned_model, plot = 'residuals')

In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.
In sample predictions has not been implemented for this estimator of type 'BaseCdsDtForecaster' in `sktime`. When this is implemented, it will be enabled by default in pycaret.


Salvando pipeline do modelo

In [73]:
save_model(tuned_model, 'pycaret_TS_completo_pipeline')

Transformation Pipeline and Model Successfully Saved


(ForecastingPipeline(steps=[('forecaster',
                             TransformedTargetForecaster(steps=[('transformer_target',
                                                                 TransformerPipeline(steps=[('numerical_imputer',
                                                                                             Imputer(method='nearest',
                                                                                                     random_state=42))])),
                                                                ('model',
                                                                 BaseCdsDtForecaster(fe_target_rr=[WindowSummarizer(lag_feature={'lag': [18,
                                                                                                                                         17,
                                                                                                                                         16,
                 

Carregando pipeline do modelo

In [ ]:
loaded_best_pipeline = load_model('pycaret_TS_completo_pipeline')
loaded_best_pipeline